In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from tensorflow.keras.layers import Input, Conv2D, Dense, Activation, Flatten, BatchNormalization, MaxPooling2D, concatenate, Dropout, Lambda, RepeatVector, GlobalAveragePooling2D
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model, Sequential
import numpy as np
import os
import tensorflow as tf
import cv2
import imutils
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental import preprocessing
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_auc_score, roc_curve, f1_score
import matplotlib.pyplot as plt

In [ ]:
!unzip /content/drive/MyDrive/DATASET.zip

Archive:  /content/drive/MyDrive/DATASET.zip
replace DATASET/TEST/IMAGES/EOSINOPHIL/_0_1616.jpeg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
image_size=(96,96)
code={"EOSINOPHIL":0,"LYMPHOCYTE":1,"MONOCYTE":2,"NEUTROPHIL":3}

images = []
vectors = []
labels = []

def findedges(image):
    gray = cv2.GaussianBlur(image, (1, 1), 0)
    edged = cv2.Canny(gray, 100, 400)
    edged = cv2.dilate(edged, None, iterations=1)
    edged = cv2.erode(edged, None, iterations=1)
    return edged

def getimageconturs(edged):
    contours = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(contours)
    contours = sorted(contours, key=lambda x: cv2.contourArea(x))
    return contours

def getboxes(contours,orig):
    boxes = []
    centers = []
    for contour in contours:
        box = cv2.minAreaRect(contour)
        box = cv2.cv.BoxPoints(box) if imutils.is_cv2() else cv2.boxPoints(box)
        box = np.array(box, dtype="int")
        (tl, tr, br, bl) = box
        if (np.linalg.norm(tl - bl)) > 0 and (np.linalg.norm(tl - tr)) > 0:
            boxes.append(box)
    return boxes

def getcode(n):
    if type(n)==str:
        for x,y in code.items():
            if n==x:
                return y
    else:
        for x,y in code.items():
            if n==y:
                return x

In [ ]:
def load_and_preprocess_data(images_path, path):

    for label, folder_path in enumerate(path):
        vec_files = os.listdir(folder_path)
        for vec_file in vec_files:
            filename = vec_file.split(".")[0]
            class_name = getcode(label)
            image_path = f"{images_path}/{class_name}/{filename}.jpeg"
            vector_path = f"{folder_path}{vec_file}"

            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # add padding to the image to better detect cell at the edge
            image = cv2.copyMakeBorder(image,10,10,10,10,cv2.BORDER_CONSTANT,value=[198, 203, 208])

            #thresholding the image to get the target cell
            image1 = cv2.inRange(image,(80, 80, 180),(180, 170, 245))

            # openning errosion then dilation
            kernel = np.ones((3, 3), np.uint8)
            kernel1 = np.ones((5, 5), np.uint8)
            img_erosion = cv2.erode(image1, kernel, iterations=2)
            image1 = cv2.dilate(img_erosion, kernel1, iterations=5)

            #detecting the blood cell
            edgedImage = findedges(image1)
            edgedContours = getimageconturs(edgedImage)
            edgedBoxes =  getboxes(edgedContours, image.copy())

            if len(edgedBoxes)==0:
                continue
            # get the large box and get its cordinate
            last = edgedBoxes[-1]
            max_x = int(max(last[:,0]))
            min_x = int( min(last[:,0]))
            max_y = int(max(last[:,1]))
            min_y = int(min(last[:,1]))

            # draw the contour and fill it
            mask = np.zeros_like(image)
            cv2.drawContours(mask, edgedContours, len(edgedContours)-1, (255,255,255), -1)

            # any pixel but the pixels inside the contour is zero
            image[mask==0] = 0

            # extract th blood cell
            image = image[min_y:max_y, min_x:max_x]

            if (np.size(image)==0):
                continue
            # resize th image

            image = cv2.resize(image, image_size)
            image = image / 255.0  # Normalize pixel values to [0, 1]

            vector = np.loadtxt(vector_path, delimiter=',')

            images.append(image)
            vectors.append(vector)
            labels.append(label)


In [ ]:
paths = [
    '/content/DATASET/TRAIN/VECTORS/EOSINOPHIL/',
    '/content/DATASET/TRAIN/VECTORS/LYMPHOCYTE/',
    '/content/DATASET/TRAIN/VECTORS/MONOCYTE/',
    '/content/DATASET/TRAIN/VECTORS/NEUTROPHIL/',
]

test_paths = [
    '/content/DATASET/TEST/VECTORS/EOSINOPHIL/',
    '/content/DATASET/TEST/VECTORS/LYMPHOCYTE/',
    '/content/DATASET/TEST/VECTORS/MONOCYTE/',
    '/content/DATASET/TEST/VECTORS/NEUTROPHIL/'
]


In [ ]:
load_and_preprocess_data("/content/DATASET/TRAIN/IMAGES", paths)

In [ ]:
load_and_preprocess_data("/content/DATASET/TEST/IMAGES", test_paths)

In [ ]:
images,vectors,labels=shuffle(images,vectors,labels,random_state=10)

In [ ]:
images = np.array(images)
vectors = np.array(vectors)
labels = np.array(labels)

In [ ]:
# Generate Train and Test using 80/20 split
train_image, test_image, train_vector, test_vector, train_label, test_label = train_test_split(images, vectors, labels, test_size=0.2, random_state=42)

In [ ]:
train_image = np.array(train_image, dtype = 'float32')
train_vector = np.array(train_vector, dtype = 'float32')
train_label = np.array(train_label, dtype = 'int32')

test_image = np.array(test_image, dtype = 'float32')
test_vector = np.array(test_vector, dtype = 'float32')
test_label = np.array(test_label, dtype = 'int32')

In [ ]:
print(train_image.shape, train_vector.shape, train_label.shape)
print(test_image.shape, test_vector.shape, test_label.shape)

(9716, 96, 96, 3) (9716, 10) (9716,)
(2430, 96, 96, 3) (2430, 10) (2430,)


In [ ]:
def create_model_without_knowledge():
    # Input layer
    image_input = Input(shape=(96, 96, 3))

    # Load the ResNet50 model without pre-trained weights
    base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=image_input)

    # Add custom top layers for fine-tuning
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.25)(x)

    # Output layer for classification (assuming 4 classes)
    output = Dense(4, activation='softmax')(x)

    # Combine base model and top layers
    model = Model(inputs=image_input, outputs=output)

    # Compile the model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

# Create and compile the ResNet50 model without pre-trained weights
resnet_model = create_model_without_knowledge()

# Display model summary
resnet_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 96, 96, 3)]          0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 102, 102, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 48, 48, 64)           9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 48, 48, 64)           256       ['conv1_conv[0][0]']          
 on)                                                                                          

In [ ]:
resnet_model.fit(train_image, train_label, validation_split=0.1, epochs=30, batch_size=32)

Epoch 1/30
274/274 [==============================] - 70s 107ms/step - loss: 0.5090 - accuracy: 0.8222 - val_loss: 2.6173 - val_accuracy: 0.2387
Epoch 2/30
274/274 [==============================] - 26s 96ms/step - loss: 0.2050 - accuracy: 0.9188 - val_loss: 1.7051 - val_accuracy: 0.2387
Epoch 3/30
274/274 [==============================] - 26s 96ms/step - loss: 0.1575 - accuracy: 0.9413 - val_loss: 4.4621 - val_accuracy: 0.2551
Epoch 4/30
274/274 [==============================] - 27s 98ms/step - loss: 0.4915 - accuracy: 0.8390 - val_loss: 0.5088 - val_accuracy: 0.8251
Epoch 5/30
274/274 [==============================] - 26s 94ms/step - loss: 0.1897 - accuracy: 0.9266 - val_loss: 0.1971 - val_accuracy: 0.9259
Epoch 6/30
274/274 [==============================] - 26s 96ms/step - loss: 0.1388 - accuracy: 0.9473 - val_loss: 0.1928 - val_accuracy: 0.9300
Epoch 7/30
274/274 [==============================] - 26s 94ms/step - loss: 0.1315 - accuracy: 0.9512 - val_loss: 0.2911 - val_accuracy

In [ ]:
loss1,accuracy1=resnet_model.evaluate(test_image, test_label)
print("the accuracy of resnet model without knowledge is : ",accuracy1)

76/76 [==============================] - 3s 39ms/step - loss: 0.1183 - accuracy: 0.9613
the accuracy of resnet model without knowledge is :  0.9613168835639954


In [ ]:
resnet_model.save('/content/drive/My Drive/ResNetPretrainedModelWithoutKnowledge.keras')

In [ ]:
resnet_model.save('/content/drive/My Drive/ResNetPretrainedModelWithoutKnowledge.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
def create_model_with_knowledge():
    # Image input for ResNet50 (resize images to 229x229)
    image_input = Input(shape=(96, 96, 3))
    base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=image_input)

    # Additional processing layers
    x = base_model.output
    x = GlobalAveragePooling2D()(x)

    # Vector input
    vector_input = Input(shape=(10,))
    y = Dense(64)(vector_input)
    y = Activation('relu')(y)
    y = Dropout(0.25)(y)

    # Concatenate image and vector features
    z = concatenate([y, x])
    z = Dense(64)(z)
    z = Activation('relu')(z)
    z = Dropout(0.25)(z)

    z = Dense(64)(z)
    z = Activation('relu')(z)
    z = Dropout(0.25)(z)

    # Output layer
    output = Dense(4, activation='softmax')(z)  # Assuming 4 classes

    # Define the model
    model = Model(inputs=[image_input, vector_input], outputs=output)

    # Compile the model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

# Create and compile the model
model_with_vector_resnet50 = create_model_with_knowledge()

# Display model summary
model_with_vector_resnet50.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 96, 96, 3)]          0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 102, 102, 3)          0         ['input_2[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 48, 48, 64)           9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 48, 48, 64)           256       ['conv1_conv[0][0]']          
 on)                                                                                        

In [ ]:
model_with_vector_resnet50.fit([train_image, train_vector], train_label, validation_split=0.1, epochs=30, batch_size=32)

Epoch 1/30
274/274 [==============================] - 59s 101ms/step - loss: 0.3846 - accuracy: 0.9092 - val_loss: 0.0418 - val_accuracy: 0.9928
Epoch 2/30
274/274 [==============================] - 25s 93ms/step - loss: 0.1040 - accuracy: 0.9796 - val_loss: 0.0411 - val_accuracy: 0.9928
Epoch 3/30
274/274 [==============================] - 26s 94ms/step - loss: 0.1107 - accuracy: 0.9585 - val_loss: 0.0800 - val_accuracy: 0.9681
Epoch 4/30
274/274 [==============================] - 25s 93ms/step - loss: 0.0986 - accuracy: 0.9692 - val_loss: 0.0632 - val_accuracy: 0.9928
Epoch 5/30
274/274 [==============================] - 26s 93ms/step - loss: 0.1019 - accuracy: 0.9708 - val_loss: 0.0462 - val_accuracy: 0.9928
Epoch 6/30
274/274 [==============================] - 25s 92ms/step - loss: 0.1023 - accuracy: 0.9694 - val_loss: 0.5580 - val_accuracy: 0.7418
Epoch 7/30
274/274 [==============================] - 25s 92ms/step - loss: 0.0790 - accuracy: 0.9808 - val_loss: 0.0769 - val_accuracy

In [ ]:
loss2,accuracy2=model_with_vector_resnet50.evaluate([test_image, test_vector], test_label)
print("the accuracy of resnet model with knowledge is : ",accuracy2)

76/76 [==============================] - 2s 27ms/step - loss: 0.0424 - accuracy: 0.9914
the accuracy of resnet model with knowledge is :  0.9913580417633057


In [ ]:
model_with_vector_resnet50.save('/content/drive/My Drive/ResNetPretrainedModelWithKnowledge.keras')

In [ ]:
model_with_vector_resnet50.save('/content/drive/My Drive/ResNetPretrainedModelWithKnowledge.h5')